## udf 등록 및 SQL 처리

In [6]:
# SparkSession 설정
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('udf').getOrCreate()

In [37]:
# 샘플 데이터 : name(이름)과 age(나이)
Age_category = [
    ('박창준', 70),
    ('최태용', 40),
    ('김보경', 38),
    ('이준석', 22),
    ('전수복', 24),
    ('정민영', 16),
    ('박준호', 82),
    ('김호진', 56),
    ('박윤철', 60),
    ('유재정', 58)
]

In [38]:
# 스키마 정의
schema = ['name', 'age']

In [39]:
# 데이터프레임 생성
df = spark.createDataFrame(data = Age_category, schema = schema)

In [40]:
# 데이터프레임 출력
df.show()

+------+---+
|  name|age|
+------+---+
|박창준| 70|
|최태용| 40|
|김보경| 38|
|이준석| 22|
|전수복| 24|
|정민영| 16|
|박준호| 82|
|김호진| 56|
|박윤철| 60|
|유재정| 58|
+------+---+



In [41]:
# 데이터프레임 스키마 출력
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [42]:
# Temporary View 생성
df.createOrReplaceTempView('Age_category')

In [43]:
# UDF : 나이 별로 분류
"""
- age가 35살 미만 : 'young'
- age가 35살-59살: 'adult'
- age가 60살 이상 : 'senior'
"""
def age_category(age):
    if age < 35:
        return 'young'
    elif 35 <= age <= 59:
        return 'adult'
    elif age >= 60:
        return 'senior'

In [44]:
# UDF 등록
spark.udf.register('age_category', age_category)

<function __main__.age_category(age)>

In [45]:
# 데이터프레임 출력 결과를 다시 한번 확인하고 밑에 SQL문 작성하기
df.show()

+------+---+
|  name|age|
+------+---+
|박창준| 70|
|최태용| 40|
|김보경| 38|
|이준석| 22|
|전수복| 24|
|정민영| 16|
|박준호| 82|
|김호진| 56|
|박윤철| 60|
|유재정| 58|
+------+---+



In [46]:
# SQL문 처리
spark.sql("SELECT name, age_category(age) AS age_category FROM Age_category").show()

+------+------------+
|  name|age_category|
+------+------------+
|박창준|      senior|
|최태용|       adult|
|김보경|       adult|
|이준석|       young|
|전수복|       young|
|정민영|       young|
|박준호|      senior|
|김호진|       adult|
|박윤철|      senior|
|유재정|       adult|
+------+------------+

